## Validation of sphere fitting (Figure 5)

We using simulations to validate the sphere fitting approach

In [20]:
# Import modules
import sys, h5py
import numpy as np
%matplotlib inline

# Import modules from src directory
sys.path.append("../src")
import plotting

In [21]:
# Simulation parameters
nsamples = 20
nsizes   = 300
nintensities = 80
shape = (nsamples, nsizes*nintensities)
pulse_length = 50e-15 #[s]
pixelsize = 110e-3 #[mm]
detector_distance = 2400 #[mm]
wavelength = 0.22621e-9 #[m]
diameter_limits = slice(35,300, None)
islice  = slice(20,80, None)
delta_hitscore = 50
hlimits = [200,3000]
max_error_center = 2
max_error_diameter = 4
max_error_intensity = 40

### 1. Loading data from file

In [22]:
filename = '../analysis/validation/simulation.h5'

In [23]:
with h5py.File(filename, 'r') as f:
    diameter_true   = f["simulation"]["diameter"][:].reshape(shape)
    intensity_true  = f["simulation"]["intensity"][:].reshape(shape)
    centerx_true    = f["simulation"]["cx"][:].reshape(shape)
    centery_true    = f["simulation"]["cy"][:].reshape(shape)
    diameter_fit    = f["results"]["diameter"][:].reshape(shape)
    intensity_fit   = f["results"]["intensity"][:].reshape(shape)
    intensity2_fit  = f["results"]["intensity_rescaled"][:].reshape(shape)
    centerx_fit     = f["results"]["cx"][:].reshape(shape)
    centery_fit     = f["results"]["cy"][:].reshape(shape)
    chisquared      = f["results"]["chisquared"][:].reshape(shape)
    hitscore_npeaks = f["results"]["npeaks"][:].reshape(shape)
    sum_model       = f["simulation"]["sum_model"][:].reshape(shape)
    sum_bg          = f["simulation"]["sum_background"][:].reshape(shape)
    sum_data        = f["simulation"]["sum_data"][:].reshape(shape)
    nr_photons_model = f["simulation"]["nr_photons_model"][:].reshape(shape)
    nr_photons_bg    = f["simulation"]["nr_photons_background"][:].reshape(shape)
    nr_photons_data  = f["simulation"]["nr_photons_data"][:].reshape(shape)

### 2. Converting to different units

In [24]:
# True Intensity in units of mJ/mu2 and W/cm2
intensity_true_Wcm2  = intensity_true
intensity_true_mJum2 = intensity_true * (1000 * pulse_length) / 1e8

# Fitted intensity in units of mJ/um2 and W/cm2
intensity_fit_Wcm2   = intensity_fit * 1e8 / (1000 * pulse_length)
intensity_fit_mJum2  = intensity2_fit

# Intensity in Nr. of photons
h = 6.62606957e-34 #Js
c = 299792458 #m/s
hc = h*c  #Jm
intensity_true_NrPhum2 = ((intensity_true_mJum2 / 1000.) * wavelength) / (hc)
intensity_fit_NrPhum2  = ((intensity_fit_mJum2 / 1000.) * wavelength) / (hc)
intensity_true_NrPh    = intensity_true_NrPhum2 * np.pi * (1e-3*diameter_true/2.)**2

# Make a grid of size vs. intensity
grid_shape = (nintensities, nsizes)
x = diameter_true[0].reshape(grid_shape)[0,:]
y = intensity_true_NrPhum2[0].reshape(grid_shape)[:,0]
X,Y = np.meshgrid(x, y)

# Intensity limits
ilimits = [y[islice.start], y[islice.stop-1]]
idetection = ilimits[0]

# Translate pixel to mrad
centerx_true_mrad = np.arctan(centerx_true * pixelsize / detector_distance) * 1000. + 0.7
centery_true_mrad = np.arctan(centery_true * pixelsize / detector_distance) * 1000. + 0.6
centerx_fit_mrad  = np.arctan(centerx_fit  * pixelsize / detector_distance) * 1000. + 0.7
centery_fit_mrad  = np.arctan(centery_fit  * pixelsize / detector_distance) * 1000. + 0.6

### 3. Computing errors (simulation vs. truth)

In [25]:
# Absolute error in center position
absolute_error_centerx_mrad = (centerx_fit_mrad - centerx_true_mrad).mean(axis=0)
absolute_error_centery_mrad = (centery_fit_mrad - centery_true_mrad).mean(axis=0)
absolute_error_centerx = (centerx_fit - centerx_true).mean(axis=0)
absolute_error_centery = (centery_fit - centery_true).mean(axis=0)
absolute_error_centerr = np.sqrt(absolute_error_centerx**2 + absolute_error_centery**2)

# Absolute error in diameter
absolute_error_diameter  = (diameter_fit - diameter_true).mean(axis=0)

# Absolute error in intensity
absolute_error_intensity_mJum2   = (intensity_fit_mJum2   - intensity_true_mJum2).mean(axis=0)
absolute_error_intensity_Wcm2    = (intensity_fit_Wcm2    - intensity_true_Wcm2).mean(axis=0)
absolute_error_intensity_NrPhum2 = (intensity_fit_NrPhum2 - intensity_true_NrPhum2).mean(axis=0)

hitscore_mean = hitscore_npeaks.mean(axis=0)
hits   = (hitscore_mean >= 600) & (diameter_true[0] > diameter_limits.start) & (intensity_true_NrPhum2[0] > idetection)
blanks = hitscore_mean <  600
middle = (hitscore_mean >= 600) & ~((diameter_true[0] > diameter_limits.start) & (intensity_true_NrPhum2[0] > idetection))

diameter_limit  = diameter_true.mean(axis=0).reshape(grid_shape) > diameter_limits.start
intensity_limit = intensity_true_NrPhum2.mean(axis=0).reshape(grid_shape) >= idetection

good_x = X[(hitscore_mean.reshape(grid_shape) > 600) & diameter_limit & intensity_limit]
good_y = Y[(hitscore_mean.reshape(grid_shape) > 600) & diameter_limit & intensity_limit]

middle_x = X[(hitscore_mean.reshape(grid_shape) > 600) & ~(diameter_limit & intensity_limit)]
middle_y = Y[(hitscore_mean.reshape(grid_shape) > 600) & ~(diameter_limit & intensity_limit)]

bad_x = X[(hitscore_mean.reshape(grid_shape) < 600)]
bad_y = Y[(hitscore_mean.reshape(grid_shape) < 600)]

intensity_true_hits   = intensity_true_NrPhum2.mean(axis=0)[hits]
intensity_true_blanks = intensity_true_NrPhum2.mean(axis=0)[blanks]
intensity_true_middle = intensity_true_NrPhum2.mean(axis=0)[middle]
intensity_fit_hits    = intensity_fit_NrPhum2.mean(axis=0)[hits]
intensity_fit_blanks  = intensity_fit_NrPhum2.mean(axis=0)[blanks]
intensity_fit_middle  = intensity_fit_NrPhum2.mean(axis=0)[middle]

diameter_true_hits   = diameter_true.mean(axis=0)[hits]
diameter_true_blanks = diameter_true.mean(axis=0)[blanks]
diameter_true_middle = diameter_true.mean(axis=0)[middle]
diameter_fit_hits    = diameter_fit.mean(axis=0)[hits]
diameter_fit_blanks  = diameter_fit.mean(axis=0)[blanks]
diameter_fit_middle  = diameter_fit.mean(axis=0)[middle]

### 4. Plotting

In [26]:
plot = plotting.Plot(fontsize=8, cols=3, rows=1, exclude=[0], border_in=0.5, aspect=1, window_width=1000, 
                     colorbar=False, legend=False, save_png=True)
plot.add_axes((0,0), 1,1, hfrac=1, padx=-0.01)
plot.title_label = 3*['']
plot.xlabel = [r'$x_{\mathrm{fit}} - x_{\mathrm{truth}}$ [px]',
               r'$d_{\mathrm{fit}} - d_{\mathrm{truth}}$ [nm]',
               r'$d_{\mathrm{truth}}$ [nm]']
plot.ylabel = [r'$y_{\mathrm{fit}} - y_{\mathrm{truth}}$ [px]',
               r'$(I^0_{\mathrm{fit}} - I^0_{\mathrm{truth}}) / I^0_{\mathrm{truth}}$',
               r'$I^0_{\mathrm{truth}}$ [photons/$\mu$m$^2$]']

plot.plotting_correlation(2, [good_x, middle_x, bad_x], [good_y, middle_y, bad_y],
                          logy=True, xlim=(0,nsizes), ylim=(5e8, 5e12), markersize=5, marker='s', alpha=1,
                          labels=[r'good', 'bad', 'not detected'],
                          color=[ 'lightgreen', 'orange', 'gray'])
plot.plotting_a_contour(2, X,Y,hitscore_mean.reshape(grid_shape),
                        (600,), colors='k', linewidths=1.5, label=False, format='%d')
plot.axes[2].axvline(diameter_limits.start + 1.5, color='k', linestyle='--')
plot.axes[2].axhline(idetection - 0.0005, color='k', linestyle='--')
plot.axes[2].text(250, 4e12, '(a)', va='top', ha='left', fontsize=10)
plot.axes[2].set_xticks([0,100,200,300])

plot.plotting_correlation(0,
                          [absolute_error_centerx[blanks],
                           absolute_error_centerx[middle],
                           absolute_error_centerx[hits]],
                          [absolute_error_centery[blanks],
                           absolute_error_centery[middle],
                           absolute_error_centery[hits]],
                          xlim=[-10,5], ylim=[-5,10], logx=False, logy=False,
                          color=['gray', 'orange', 'lightgreen'],
                          markersize=3, marker='o', alpha=0.5)
plot.axes[0].tick_params(axis='y', right='on', left='on', labelright='off', labelleft='on', labelcolor='k', direction='in')
plot.axes[0].text(-9.5, 9.5, '(b)', va='top', ha='left', fontsize=10)

plot.plotting_correlation(1,
                          [absolute_error_diameter[blanks],
                           absolute_error_diameter[middle],
                           absolute_error_diameter[hits]],
                          [absolute_error_intensity_NrPhum2[blanks] / intensity_true_blanks,
                           absolute_error_intensity_NrPhum2[middle] / intensity_true_middle,
                           absolute_error_intensity_NrPhum2[hits] / intensity_true_hits],
                          logx=False, logy=False, xlim=[-6,6], ylim=[-.5, 1.5],
                          color=['gray', 'orange', 'lightgreen'],
                          markersize=3, marker='o', alpha=0.5)
#plot.axes[1].tick_params(axis='y', right='on', left='on', labelright='on', labelleft='off', labelcolor='k')
#plot.axes[1].yaxis.set_label_position("right")
plot.axes[1].text(-5.5, 1.45, '(c)', va='top', ha='left', fontsize=10)

plot.save('/Users/benedikt/phd-project/documentation/manuscripts/omrv-paper/manuscript/figures/fig_simulation_simple.png') 
plot.show()

**Figure 5.**
Validation of the classification procedure based on simulation of spheres with different particle sizes and photon intensities.                                           
(a) Separation of non-hits (gray area) and hits (above the black solid line) as function of particle size and intensity.                                                           
Data points with strong deviations are depicted in orange, the rest is shown in green and separated by black dashed lines.                                                         
(b) Distribution errors in diffraction center. (c) Distribution of errors in particle size and intensity (normalized to ground truth of intensity).                                
A statistical summary of the green distributions is given below.

In [68]:
print "x, std = ", np.std(absolute_error_centerx[hits])                                                                                                                           
print "y, std = ", np.std(absolute_error_centery[hits])                                                                                                                           
print "d, std = ", np.std(absolute_error_diameter[hits])                                                                                                                          
print "I, std = ", np.std(absolute_error_intensity_NrPhum2[hits] / intensity_true_hits)                                                                                     
                                                                                                                                                                                  
print "x, min = ", np.min(absolute_error_centerx[hits])                                                                                                                           
print "y, min = ", np.min(absolute_error_centery[hits])                                                                                                                           
print "d, min = ", np.min(absolute_error_diameter[hits])                                                                                                                          
print "I, min = ", np.min(absolute_error_intensity_NrPhum2[hits] / intensity_true_hits)                                                                                     
                                                                                                                                                                                  
print "x, max = ", np.max(absolute_error_centerx[hits])                                                                                                                           
print "y, max = ", np.max(absolute_error_centery[hits])                                                                                                                           
print "d, max = ", np.max(absolute_error_diameter[hits])                                                                                                                          
print "I, max = ", np.max(absolute_error_intensity_NrPhum2[hits] / intensity_true_hits)  

x, std =  0.0948476984557
y, std =  0.0810889166573
d, std =  0.147354404899
I, std =  0.0488678594065
x, min =  -1.45763483549
y, min =  -0.586941577885
d, min =  -0.913126414295
I, min =  -0.0065876016447
x, max =  1.05037254708
y, max =  1.98510603377
d, max =  3.18761118119
I, max =  0.413115114905
